In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tools import *
import math
pd.set_option("display.max_colwidth",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.options.display.precision = 5
plt.rcParams.update(plt.rcParamsDefault)
sns.set_style('white')
import cv2

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import datetime
from time import process_time

# Extra library imports
from PIL import Image
from scipy.optimize import minimize
import scipy.ndimage as ndi
from scipy.stats import skew, kurtosis

In [ ]:
path = 'train/'
path2 = 'data/'
data = pd.read_excel(path2 + "classif.xlsx",header=0,index_col="ID")
print(data.shape)
data.head(10)

(250, 2)


,bug type,species
ID,,
1,Bee,Apis mellifera
2,Bee,Apis mellifera
3,Bee,Apis mellifera
4,Bee,Apis mellifera
5,Bee,Apis mellifera
6,Bee,Apis mellifera
7,Bee,Apis mellifera
8,Bee,Apis mellifera
9,Bee,Apis mellifera


In [ ]:
print('Dimensions: ', data.shape)

Dimensions:  (250, 2)


In [ ]:
data.dtypes

bug type    object
species     object
dtype: object

In [ ]:
get_missing(data)

,Nom de colonne,Pourcentage de NA
bug type,bug type,0.0
species,species,0.0


In [ ]:
data.rename(axis=1, mapper={'bug type':'bug_type'}, inplace=True)

In [ ]:
import os
from PIL import Image

# Load the input image
#os.path.join(os.getcwd() + path) 
image_files = [file for file in os.listdir(path) if file.endswith(('.JPG'))]
print('Found {0} images'.format(len(image_files)))



im_dict = {}

for file in image_files:
    image_path = os.path.join(path, file)
    image = Image.open(image_path)
    im_arr = np.array(image)
    sx, sy, nb_channels = im_arr.shape
    print('Loaded image {0}'.format(file))
    print('Input image of dimension {0} x {1} x {2} pixels'.format(sx, sy,
                                                                nb_channels))
    print('Input image of type {0}'.format(type(im_arr[0, 0, 0])))
    print('Minimal intensity value: {0}'.format(np.min(im_arr)))
    print('Maximal intensity value: {0}'.format(np.max(im_arr)))
    file_name = int(file[:-4])
    im_dict[file_name] = im_arr


Found 250 images
Loaded image 1.JPG
Input image of dimension 4000 x 6000 x 3 pixels
Input image of type <class 'numpy.uint8'>
Minimal intensity value: 0
Maximal intensity value: 255
Loaded image 10.JPG
Input image of dimension 4000 x 6000 x 3 pixels
Input image of type <class 'numpy.uint8'>
Minimal intensity value: 0
Maximal intensity value: 255
Loaded image 100.JPG
Input image of dimension 4000 x 6000 x 3 pixels
Input image of type <class 'numpy.uint8'>
Minimal intensity value: 0
Maximal intensity value: 255
Loaded image 101.JPG
Input image of dimension 4000 x 6000 x 3 pixels
Input image of type <class 'numpy.uint8'>
Minimal intensity value: 0
Maximal intensity value: 255
Loaded image 102.JPG
Input image of dimension 4000 x 6000 x 3 pixels
Input image of type <class 'numpy.uint8'>
Minimal intensity value: 0
Maximal intensity value: 255
Loaded image 103.JPG
Input image of dimension 4000 x 6000 x 3 pixels
Input image of type <class 'numpy.uint8'>
Minimal intensity value: 0
Maximal inten

In [ ]:
mask_path = path + 'masks/'
mask_dict = {}
mask_files = [file for file in os.listdir(mask_path) if file.endswith(('.tif'))]

for file in mask_files:
    mask_path2 = os.path.join(mask_path, file)
    mask = Image.open(mask_path2).convert('L')
    mask_arr = np.array(mask)
    #mask.show()
    #print(np.unique(mask_arr.flatten()))

    #Some values being a little over 0 or a little below 255, we need to threshold the mask
    mask_arr[mask_arr < 122] = 0
    mask_arr[mask_arr >= 122] = 1
    
    sx, sy = mask_arr.shape
    print('Loaded mask {0}'.format(file))
    print('Mask of dimension {0} x {1} pixels'.format(sx, sy))

    file_name = file[7:-4]
    print("File name : ", file_name)
    num = int(file_name)
    mask_dict[num] = mask_arr

Loaded mask binary_1.tif
Mask of dimension 4000 x 6000 pixels
File name :  1
Loaded mask binary_10.tif
Mask of dimension 4000 x 6000 pixels
File name :  10
Loaded mask binary_100.tif
Mask of dimension 4000 x 6000 pixels
File name :  100
Loaded mask binary_101.tif
Mask of dimension 4000 x 6000 pixels
File name :  101
Loaded mask binary_102.tif
Mask of dimension 4000 x 6000 pixels
File name :  102
Loaded mask binary_103.tif
Mask of dimension 4000 x 6000 pixels
File name :  103
Loaded mask binary_104.tif
Mask of dimension 4000 x 6000 pixels
File name :  104
Loaded mask binary_105.tif
Mask of dimension 4000 x 6000 pixels
File name :  105
Loaded mask binary_106.tif
Mask of dimension 4000 x 6000 pixels
File name :  106
Loaded mask binary_107.tif
Mask of dimension 4000 x 6000 pixels
File name :  107
Loaded mask binary_108.tif
Mask of dimension 4000 x 6000 pixels
File name :  108
Loaded mask binary_109.tif
Mask of dimension 4000 x 6000 pixels
File name :  109
Loaded mask binary_11.tif
Mask of 

In [ ]:
def convert_to_grayscale(im_arr):
    # Calculate the grayscale values using the standard luminosity method
    gray_im_arr = 0.2989 * im_arr[:, :, 0] + 0.5870 * im_arr[:, :, 1] + 0.1140 * im_arr[:, :, 2]
    return gray_im_arr

In [ ]:
def find_edges(mask):
    ### THIS IS THE FUNCTION USED, THE OTHER find_edges FUNCTIONS ARE NOT USED
    ### IDENTIFY THE BOUNDARY PIXELS (EDGES) OF THE INSECT
    start = process_time()
    # Shift the mask in all directions and compare with the original to detect boundaries
    shifted_up = np.roll(mask, -1, axis=0)
    shifted_down = np.roll(mask, 1, axis=0)
    shifted_left = np.roll(mask, -1, axis=1)
    shifted_right = np.roll(mask, 1, axis=1)
    
    # Handle boundary conditions by assuming outside of the mask is all zeros
    shifted_up[-1, :] = 0
    shifted_down[0, :] = 0
    shifted_left[:, -1] = 0
    shifted_right[:, 0] = 0
    
    # Identify edges by looking for discrepancies between the mask and its shifted versions
    edges = (mask != shifted_up) | (mask != shifted_down) | \
            (mask != shifted_left) | (mask != shifted_right)

    # Optionally, convert boolean mask to integer if needed
    edges = edges.astype(int)
    end = process_time()
    print("Time taken for finding edges:", end - start)
    return edges

def find_edges2(mask):
    ### DOESN'T WORK (NOT USED)
    edges = np.zeros_like(mask)
    # Check for transitions from 1 to 0 (insect to background)
    for i in range(mask.shape[0] - 1):
        for j in range(mask.shape[1] - 1):
            if mask[i, j] == 1 and (mask[i+1, j] == 0 or mask[i, j+1] == 0):
                edges[i, j] = 1
    return edges

def find_edges3(mask_arr):
    ### DOESN'T WORK (NOT USED)
    # This function will extract the boundary of the insect based on the mask
    edges = np.zeros_like(mask_arr)
    # Loop over each pixel and check for edges
    for i in range(1, mask_arr.shape[0]-1):
        for j in range(1, mask_arr.shape[1]-1):
            if mask_arr[i, j] == 1 and np.any(mask_arr[i-1:i+2, j-1:j+2] == 0):
                edges[i, j] = 1
    return edges

from skimage.segmentation import find_boundaries
def find_edges_skimage(mask):
    ### FOR TESTING PURPOSES (NOT USED)
    edges = find_boundaries(mask, mode='outer').astype(np.uint8)
    return edges 

def find_edges_opencv(mask):
    ### FOR TESTING PURPOSES (NOT USED)
    # Ensure the mask is of the correct type (uint8), and binarize just in case
    mask = np.uint8(mask * 255)
    # Find contours using OpenCV
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Create an empty image to draw contours
    #edges = np.zeros_like(mask)
    #cv2.drawContours(edges, contours, -1, (255), 1)  # Draw contours with white color
    return edges


In [ ]:
def plot_edges(im_arr,edge_points,title):
    fig, ax = plt.subplots()
    ax.imshow(im_arr, cmap='gray')
    edges_y, edges_x = zip(*edge_points)
    ax.scatter(edges_x, edges_y, color='red', s=0.2)
    ax.set_title(title)
    plt.show()

## 1 - Required features given by instructions

In [ ]:
def get_symmetry_index(gray_im_arr):
    ### COMPUTE THE SYMMETRY INDEX OF THE IMAGE
    ### SYMMETRY INDEX : THE MEAN ABSOLUTE DIFFERENCE BETWEEN THE LEFT AND RIGHT HALVES : POSSIBLE RANGE : 0 TO 255
    ### (COMPUTED FOR THE BOUNDING BOX OF THE BUG AND FOR THE WHOLE IMAGE SEPARATELY)
    ### RETURNS THE SYMMETRY INDEX
    
    start = process_time()
    print("Starting computation of symmetry index")
    # Split the grayscale image into left and right halves
    height, width = gray_im_arr.shape
    center = width // 2
    left_half = gray_im_arr[:, :center]
    right_half = gray_im_arr[:, center:]
    
    # Make sure both halves have the same width
    if right_half.shape[1] != left_half.shape[1]:
        # If the image width is odd, the right half will have one more column
        right_half = right_half[:, :-1]

    # Mirror the right half horizontally to compare with the left
    mirrored_right = np.fliplr(right_half)

    # Compute the difference between the mirrored right half and the left half
    differences = np.abs(mirrored_right - left_half)

    # Calculate the symmetry index as the mean of the differences
    symmetry_index = np.mean(differences)
    end = process_time()
    print("Computation took ", end - start)
    return symmetry_index

In [ ]:
def calculate_color_aggregates(row,channel,channel_arr):
    ### COMPUTE THE MEAN, MEDIAN, STANDARD DEVIATION, MINIMUM, MAXIMUM, 1ST QUARTILE AND 3RD QUARTILE OF THE COLOR CHANNEL
    ### ORIGINALLY : ONLY THE MEAN, MIN AND MAX WERE REQUIRED
    ### WE DECIDED TO ADD THE REST FOR MORE INFORMATION 
    ### (COMPUTED FOR BUG AND NON BUG POINTS SEPARATELY)
    row[channel + "_mean"] = np.mean(channel_arr)
    row[channel + "_median"] = np.median(channel_arr)
    row[channel + "_std"] = np.std(channel_arr)
    row[channel + "_min"] = np.min(channel_arr)
    row[channel + "_max"] = np.max(channel_arr)
    row[channel + "_q1"] = np.quantile(channel_arr, .25)
    row[channel + "_q3"] = np.quantile(channel_arr, .75)

In [ ]:
from sklearn.decomposition import PCA

def find_and_plot_orthogonal_lines(mask, plot=False):
    ### APPROXIMATION OF THE RATIO OF THE LONGEST ORTHOGONAL LINES THAT CAN BE FITTED INSIDE THE MASK USING PCA
    ### COULD NOT FIND A BETTER WAY TO DO THIS

    points = np.column_stack(np.where(mask > 0))  # Get indices of non-zero elements, which are points of the insect

    if len(points) < 5:
        print("Not enough points to fit an ellipse")
        return 0

    # Fit PCA on the provided points
    pca = PCA(n_components=2)
    pca.fit(points)

    # Calculate the center and directions of the principal components
    center = pca.mean_
    direction1 = pca.components_[0]
    direction2 = pca.components_[1]

    # Extend lines to the edge of the mask by finding intersections
    def find_edge(center, direction, mask):
        t_max = 1000  # Maximum line length to check
        for t in np.linspace(0, t_max, num=int(t_max)):
            point = center + direction * t
            if not (0 <= int(point[0]) < mask.shape[0] and 0 <= int(point[1]) < mask.shape[1]) or mask[int(point[0]), int(point[1])] == 0:
                return center + direction * (t - 1)  # Step back one unit
        return center + direction * t_max

    line1_point1 = find_edge(center, -direction1, mask)
    line1_point2 = find_edge(center, direction1, mask)
    line2_point1 = find_edge(center, -direction2, mask)
    line2_point2 = find_edge(center, direction2, mask)

    # Calculate the ratio of the smaller dimension to the larger dimension
    length1 = np.linalg.norm(line1_point2 - line1_point1)
    length2 = np.linalg.norm(line2_point2 - line2_point1)
    ratio = min(length1, length2) / max(length1, length2) if max(length1, length2) != 0 else 0

    if plot:
        # Plotting
        fig, ax = plt.subplots()
        ax.imshow(mask, cmap='gray', interpolation='nearest')
        ax.plot([line1_point1[1], line1_point2[1]], [line1_point1[0], line1_point2[0]], 'r-')  # Red line
        ax.plot([line2_point1[1], line2_point2[1]], [line2_point1[0], line2_point2[0]], 'b-')  # Blue line
        plt.axis('off')
        plt.show()

    return ratio


### Functions for additional features (our own)

In [ ]:
def get_ratio(mask_area,im_1d_arr):
    ### COMPUTE THE RATIO BETWEEN THE AREA OF THE INSECT AND THE TOTAL AREA OF THE IMAGE (nb_pixels_ratio)
    ratio = mask_area / np.size(im_1d_arr)
    #print('The bug fills {0}% of the image'.format(ratio * 100))
    return ratio

In [ ]:
def calculate_bounding_rectangle_features(row, mask_arr, mask_area):
    ### COMPUTE THE ASPECT RATIO AND RECTANGULARITY OF THE INSECT BY FINDING THE BOUNDING BOX
    ### ASPECT RATIO = MIN(HEIGHT, WIDTH) / MAX(HEIGHT, WIDTH)
    ### RECTANGULARITY = AREA OF THE INSECT / (HEIGHT * WIDTH)
    ### RETURN THE TOP, BOTTOM, LEFT AND RIGHT BOUNDARIES OF THE BOUNDING BOX

    start = process_time()
    print("Starting computation of bounding rectangle features")
    # Ensure that mask_arr is a binary mask (0s and 1s)
    if mask_arr.dtype != bool:
        mask_arr = mask_arr > 0

    # Find rows and columns where the mask has at least one pixel
    rows_with_insect = np.any(mask_arr, axis=1)
    cols_with_insect = np.any(mask_arr, axis=0)

    # Find the indices of the rows and columns that contain the insect
    rows_indices = np.where(rows_with_insect)[0]
    cols_indices = np.where(cols_with_insect)[0]

    if len(rows_indices) == 0 or len(cols_indices) == 0:
        # No insect found in the mask, return an aspect ratio of 0
        return 0

    # Calculate the max extents in both directions
    vertical_extent = rows_indices[-1] - rows_indices[0] + 1
    horizontal_extent = cols_indices[-1] - cols_indices[0] + 1

    # Calculate the aspect ratio (smaller dimension divided by the larger dimension)
    aspect_ratio = min(vertical_extent, horizontal_extent) / max(vertical_extent, horizontal_extent)

    end = process_time()
    print("Computation took ", end - start)
    row['aspect_ratio'] = aspect_ratio
    row['rectangularity'] = mask_area / (vertical_extent * horizontal_extent)

    # Extract the mask values within the bounding box dimensions
    return rows_indices[0],rows_indices[-1], cols_indices[0],cols_indices[-1]

In [ ]:
def get_roundness(area, edges):
    ### COMPUTE THE ROUNDNESS USING AREA AND PERIMETER : INDICATOR OF HOW ROUND THE MASK OF THE BUG IS
    ### ROUNDNESS = 4 * PI * AREA / PERIMETER^2 (PERIMETER : SUM OF EDGE PIXELS ; AREA : SUM OF MASK PIXELS)
    ### RETURN THE ROUNDNESS AND PERIMETER OF THE MASK
    
    perimeter = np.sum(edges)

    if perimeter == 0:  # Avoid division by zero
        return 0
    roundness = 4 * np.pi * area / (perimeter ** 2)
    return roundness, perimeter

def get_roundness2(mask,area):
    ### OLD FUNCTION (NOT USED)
    
    # Calculate the perimeter by detecting edges within the mask
    # Shift the mask in all directions and compare with the original to detect boundaries
    shifted_up = np.roll(mask, -1, axis=0)
    shifted_down = np.roll(mask, 1, axis=0)
    shifted_left = np.roll(mask, -1, axis=1)
    shifted_right = np.roll(mask, 1, axis=1)
    
    # Handle boundary conditions by assuming outside of the mask is all zeros
    shifted_up[-1, :] = 0
    shifted_down[0, :] = 0
    shifted_left[:, -1] = 0
    shifted_right[:, 0] = 0
    
    # Calculate perimeter by summing up the boundary detections
    perimeter = np.sum(mask != shifted_up) + np.sum(mask != shifted_down) + \
                np.sum(mask != shifted_left) + np.sum(mask != shifted_right)
    print("Calculated perimeter is ", perimeter)
    
    # Calculate roundness using the area and perimeter
    if perimeter == 0:  # To avoid division by zero error
        return 0, 0
    roundness = 4 * np.pi * area / (perimeter ** 2)

    return roundness, perimeter

In [ ]:
def rgb_to_hsv_opencv(rgb_image):
    ### TOOL TO CONVERT RED GREEN BLUE TO HUE SATURATION VALUE WITH OPENCV
    start = process_time()
    print("Starting conversion from RGB to HSV using OPENCV")
    hsv_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)
    if hsv_image.shape[0] != rgb_image.shape[0]:
        # Transpose only if the height is greater than the width
        hsv_image = np.transpose(hsv_image, (1, 0, 2))
    end = process_time()
    print("The conversion took ", end-start)
    return hsv_image

def rgb_to_hsv(rgb_image):
    ### (UNUSED) TOOL TO CONVERT RED GREEN BLUE TO HUE SATURATION VALUE MANUALLY 
    ### HUE = 60 * (0 + (G - B) / delta) % 360 (delta : c_max - c_min ; c_max : max of R,G,B ; c_min : min of R,G,B)
    ### SATURATION = delta / c_max 
    ### VALUE = c_max
    start = process_time()
    print("Starting conversion from RGB to HSV ")
    rgb_image = rgb_image.astype('float') / 255  # Normalize RGB values to [0, 1]

    # Prepare arrays for the HSV channels
    hsv_image = np.zeros_like(rgb_image)

    r, g, b = rgb_image[:, :, 0], rgb_image[:, :, 1], rgb_image[:, :, 2]
    c_max = np.max(rgb_image, axis=2)
    c_min = np.min(rgb_image, axis=2)
    delta = c_max - c_min

    # Initialize hue
    hue = np.zeros_like(c_max)

    # Calculate the hue component
    mask = delta != 0  # Mask where delta is not zero
    # Red is max
    hue[mask & (c_max == r)] = 60 * (0 + (g - b)[mask & (c_max == r)] / delta[mask & (c_max == r)]) % 360
    # Green is max
    hue[mask & (c_max == g)] = 60 * (2 + (b - r)[mask & (c_max == g)] / delta[mask & (c_max == g)]) % 360
    # Blue is max
    hue[mask & (c_max == b)] = 60 * (4 + (r - g)[mask & (c_max == b)] / delta[mask & (c_max == b)]) % 360

    # Calculate the saturation component
    saturation = np.zeros_like(c_max)
    saturation[c_max != 0] = delta[c_max != 0] / c_max[c_max != 0]

    # Value component is just the max of R, G, B
    value = c_max

    # Combine into a single HSV image
    hsv_image[:, :, 0] = hue
    hsv_image[:, :, 1] = saturation
    hsv_image[:, :, 2] = value

    end = process_time()
    print("The conversion took ", end-start)
    return hsv_image

In [ ]:
def calculate_hsv_statistics_for_array(row,channel_name,arr):
    ### COMPUTE THE MEAN, MEDIAN, STANDARD DEVIATION, MINIMUM, MAXIMUM, 1ST QUARTILE AND 3RD QUARTILE OF THE HSV CHANNEL
    ### (COMPUTED FOR BUG AND NON BUG POINTS SEPARATELY)
    row[f'{channel_name }_mean'] = np.mean(arr)
    row[f'{channel_name}_std'] = np.std(arr)
    row[f'{channel_name}_median'] = np.median(arr)
    row[f'{channel_name}_min'] = np.min(arr)
    row[f'{channel_name}_max'] = np.max(arr)
    row[f'{channel_name}_q1'] = np.quantile(arr, .25)
    row[f'{channel_name}_q3'] = np.quantile(arr, .75)

def calculate_hsv_statistics(row, hsv_image, bug_indices, non_bug_indices):
    ### COMPUTE THE HSV STATISTICS 
    ### USED FOR BOTH THE BUG AND NON BUG PIXELS
    start = process_time()
    print("Starting computation of HSV statistics ")

    # Calculate statistics for each channel
    for i, channel_name in enumerate(['hue', 'saturation', 'value']):
        channel_data = hsv_image[:, :, i]
        
        calculate_hsv_statistics_for_array(row,channel_name + '_mask',channel_data[bug_indices])
        calculate_hsv_statistics_for_array(row,channel_name + '_rest',channel_data[non_bug_indices])

    end = process_time()
    print("Computation took ", end - start)

In [ ]:
def calculate_entropy(gray_im_arr):
    ### COMPUTE THE ENTROPY OF THE IMAGE TEXTURE : INDICATOR OF RANDOMNESS (COMPLEXITY) IN THE IMAGE TEXTURE 
    ### ENTROPY = -SUM(p * log(p)) WHERE p IS THE PROBABILITY OF EACH INTENSITY LEVEL
    ### (COMPUTED FOR BOTH BUG AND NON BUG PIXELS : MASK AND REST)
    start = process_time()
    print("Starting computation of entropy")

    # Calculate histogram of pixel intensities in the masked region
    histogram, _ = np.histogram(gray_im_arr, bins=256, range=(0, 256))
    histogram = histogram.astype(float)

    # Normalize the histogram to get probability distribution
    histogram /= histogram.sum()

    # Calculate entropy using the entropy formula: -sum(p * log(p)) where p is the probability of each intensity level
    with np.errstate(divide='ignore', invalid='ignore'):
        # Ignore log(0) which is undefined; np.where takes care of zero probabilities
        entropy = -np.sum(np.where(histogram > 0, histogram * np.log2(histogram), 0))
    
    end = process_time()
    print("Computation took ", end - start)
    return entropy

In [ ]:
def calculate_geometric_centroid_distance_aggregates(row,mask_arr, bug_argwhere_indices, centroid, boundary_pixels=None, plot=None):
    ### COMPUTE THE DISTANCES OF EACH BOUNDARY PIXEL FROM THE GEOMETRIC CENTROID OF THE INSECT (mean...)
    ### CENTROID DISTANCE = SQRT((Y - Yc)^2 + (X - Xc)^2) WHERE (Yc, Xc) IS THE CENTROID
    ### THEN COMPUTE STATISTICS ON THESE DISTANCES (mean, std, min, max, q1, q3, skewness, kurtosis)  
    
    start = process_time()
    print("Starting computation of geometric centroid distance aggregates")
    
    # Identify boundary pixels using a simple edge detection logic
    if boundary_pixels is None:
        boundary_pixels = []
        for y, x in bug_argwhere_indices:
            # Check if any of the eight surrounding pixels are zero (part of the background)
            if (mask_arr[y-1:y+2, x-1:x+2] == 0).any():
                boundary_pixels.append((y, x))
    if plot is not None:
        print(len(boundary_pixels))
        plot_edges(mask_arr,boundary_pixels,plot)

    # Calculate distances from centroid to each boundary pixel
    boundary_pixels = np.array(boundary_pixels)
    distances = np.linalg.norm(boundary_pixels - centroid, axis=1)
    
    # Compute statistics
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    max_distance = np.max(distances)
    q1_distance = np.quantile(distances,0.25)
    q3_distance = np.quantile(distances,0.75)
    min_distance = np.min(distances)
    skewness = skew(distances)
    kurt = kurtosis(distances)
    row['mean_centroid_distance'], row['std_centroid_distance'], row['min_centroid_distance'], row['max_centroid_distance'] = mean_distance, std_distance, min_distance, max_distance
    row['skewness_centroid_distance'], row['kurtosis_centroid_distance'] = skewness, kurt
    row['q1_centroid_distance'], row['q3_centroid_distance'] = q1_distance, q3_distance

    end = process_time()
    print("Computation took ", end - start)

In [ ]:
def calculate_fractal_dimension(mask_arr, edges):
    ### UNUSED (DOESNT WORK)
    start = process_time()
    print("Starting computation of fractal dimension")

    # Calculate the fractal dimension using the box-counting method
    def box_counting(edges):
        sizes = np.arange(1, min(edges.shape)//2, 2)  # range of box sizes
        counts = []
        for size in sizes:
            count = 0
            for i in range(0, edges.shape[0], size):
                for j in range(0, edges.shape[1], size):
                    block = edges[i:i+size, j:j+size]
                    if np.any(block == 1):
                        count += 1
            counts.append(count)
        
        counts = np.array(counts)
        sizes = np.array(sizes)
        # Use only sizes and counts that are non-zero to avoid log(0)
        valid = counts > 0
        coeffs = np.polyfit(np.log(sizes[valid]), np.log(counts[valid]), 1)
        return -coeffs[0]  # Fractal dimension is the negative slope of log-log plot

    fractal_dimension = box_counting(edges)
    end = process_time()
    print("Computation took ", end - start)
    return fractal_dimension

from skimage.measure import label, regionprops
from skimage.transform import resize
def calculate_fractal_dimension_fast(thresholded):
    ### UNUSED (DOESNT WORK)
    start = process_time()
    print("Starting computation of fractal dimension")

    # List to hold the number of boxes and scales
    scales = np.linspace(0.1, 1.0, num=10)
    counts = []

    # Analyze fractal dimension using different scales
    for scale in scales:
        scaled = resize(thresholded, (int(scale * image.shape[0]), int(scale * image.shape[1])), order=0, preserve_range=True, anti_aliasing=False)
        labels = label(scaled)
        properties = regionprops(labels)
        # Count regions with area larger than 0
        count = sum([prop.area for prop in properties if prop.area > 0])
        counts.append(count)

    # Calculate the fractal dimension
    # Assuming counts vary with scale as a power law (which is typical for fractal objects)
    coeffs = np.polyfit(np.log(scales), np.log(counts), 1)
    fractal_dim = -coeffs[0]
    end = process_time()
    print("Computation took ", end - start)


In [ ]:
def calculate_fourier_descriptors(row,mask_arr, num_descriptors, edges_x, edges_y):
    ### COMPUTE THE FOURIER DESCRIPTORS OF THE INSECT BOUNDARY (EDGES) TO DESCRIBE THE SHAPE OF THE INSECT
    ### FOURIER DESCRIPTOR : A COMPLEX NUMBER REPRESENTING THE AMPLITUDE AND PHASE OF A WAVE
    
    start = process_time()
    print("Starting computation of Fourier descriptors")
    # Combine x and y to form a complex number, which represents the boundary
    boundary_complex = edges_x + 1j * edges_y
    # Compute the Fourier Transform
    fourier_result = np.fft.fft(boundary_complex)
    # Retain only the first 'num_descriptors' Fourier coefficients
    descriptors = fourier_result[:num_descriptors]
    # Normalize the descriptors by the first descriptor to make the description scale-invariant
    descriptors /= np.abs(descriptors[0])
    end = process_time()
    print("Computation took ", end - start)
    for i, descriptor in enumerate(descriptors):
        row[f'fourier_descriptor_real_{i}'] = descriptor.real
        row[f'fourier_descriptor_imag_{i}'] = descriptor.imag
        row
    return descriptors

In [ ]:
def calculate_convex_hull_properties(row, mask_arr, hull, insect_area, insect_perimeter):
    ### COMPUTE THE PROPERTIES OF THE CONVEX HULL (SMALLEST CONVEX SHAPE THAT ENCLOSES THE INSECT) BY COMPARING IT TO THE INSECT
    ### HULL TO INSECT AREA RATIO : RATIO OF THE AREA OF THE CONVEX HULL TO THE AREA OF THE INSECT
    ### HULL CONVEXITY : RATIO OF THE PERIMETER OF THE CONVEX HULL TO THE PERIMETER OF THE INSECT
    hull_area = hull.volume  # In 2D, volume is the area
    hull_perimeter = hull.area  # In 2D, area is the perimeter

    # Area ratio (insect area to convex hull area)
    area_ratio = insect_area / hull_area if hull_area > 0 else 0

    # Convexity
    convexity = hull_perimeter / insect_perimeter if hull_perimeter > 0 else 0
    
    row['hull_area'] = hull_area
    row['hull_to_insect_area_ratio'] = area_ratio
    row['hull_convexity'] = convexity


In [ ]:
def fit_triangle(points, hull):
    ### (USED BY BELOW FUNCTION get_triangle_similarity FOR TRIANGLE SIMILARITY SCORE) 
    ### COMPUTE THE TRIANGLE THAT FITS THE SHAPE OF THE INSECT 
    ### RETURN THE TRIANGLE AND THE AREA OF THE TRIANGLE

    start = process_time()
    print("Starting computation of triangle fit")

    hull_points = points[hull.vertices]

    # Simplified assumption: Pick three points that are furthest apart in the hull to form a triangle
    max_dist = 0
    triangle = None
    n = len(hull_points)
    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                tri_points = np.array([hull_points[i], hull_points[j], hull_points[k]])
                # Calculate perimeter of the triangle
                perimeter = np.sum(np.sqrt(np.sum(np.diff(np.vstack([tri_points, tri_points[0]]), axis=0)**2, axis=1)))
                if perimeter > max_dist:
                    max_dist = perimeter
                    triangle = tri_points

    # Calculate area of the triangle using Heron's formula
    if triangle is not None:
        a = np.linalg.norm(triangle[0] - triangle[1])
        b = np.linalg.norm(triangle[1] - triangle[2])
        c = np.linalg.norm(triangle[2] - triangle[0])
        s = 0.5 * (a + b + c)
        area = math.sqrt(s * (s - a) * (s - b) * (s - c))
        end = process_time()
        print("Computation took ", end - start)
        return triangle, area
    end = process_time()
    print("Computation took ", end - start)
    return None, 0


In [ ]:
def get_triangle_similarity(hull, points, insect_area):    
    ### COMPUTE THE SIMILARITY SCORE BETWEEN THE INSECT AND THE TRIANGLE THAT FITS THE SHAPE OF THE INSECT
    ### SIMILARITY = MIN(AREA OF INSECT, AREA OF TRIANGLE) / MAX(AREA OF INSECT, AREA OF TRIANGLE)
    
    # Fit triangle to points
    triangle, triangle_area = fit_triangle(points, hull)
    
    # Similarity score based on area comparison
    if triangle_area == 0:  # Avoid division by zero
        return 0
    similarity = min(insect_area, triangle_area) / max(insect_area, triangle_area)
    return similarity

In [ ]:
def calculate_ellipse_features(row, mask_arr, points):
    ### COMPUTE THE FEATURES OF THE ELLIPSE FIT TO THE INSECT USING THE POINTS OF THE INSECT
    ### ELLIPSE ANGLE = ANGLE OF THE ELLIPSE
    ### ELLIPSE AXIS RATIO = MINOR AXIS LENGTH / MAJOR AXIS LENGTH
    ### ELLIPSE ECCENTRICITY FORMULA = SQRT(1 - (b^2 / a^2)) WHERE a = MAJOR AXIS LENGTH AND b = MINOR AXIS LENGTH

    start = process_time()
    print("Starting computation of ellipse features")
    
    # Check if there are enough points to fit an ellipse
    if len(points) < 5:
        print("Not enough points to fit an ellipse")
        return
    
    # Fit an ellipse to the points
    ellipse = cv2.fitEllipse(points)
    
    # Extract features from the ellipse
    (center, axes, angle) = ellipse
    major_axis_length = max(axes)
    minor_axis_length = min(axes)
    axis_ratio = minor_axis_length / major_axis_length if major_axis_length != 0 else 0
    
    # Calculate eccentricity
    a = major_axis_length / 2  # semi-major axis
    b = minor_axis_length / 2  # semi-minor axis
    eccentricity = np.sqrt(1 - (b**2 / a**2)) if a != 0 else 0

    # Calculate ellipse variance
    ellipse_variance = calculate_ellipse_variance(points, ellipse)

    row['ellipse_angle'] = angle
    row['ellipse_axis_ratio'] = axis_ratio
    row['ellipse_eccentricity'] = eccentricity
    row['ellipse_variance'] = ellipse_variance

    end = process_time()
    print("Computation took ", end - start)

def calculate_ellipse_variance(points, ellipse):
    (center, axes, angle) = ellipse
    cx, cy = center
    rx, ry = axes[0] / 2, axes[1] / 2
    angle_rad = np.deg2rad(angle)
    cos_angle = np.cos(angle_rad)
    sin_angle = np.sin(angle_rad)

    def point_to_ellipse_distance(x, y):
        # Translate point to origin based on ellipse center
        xt = x - cx
        yt = y - cy
        
        # Rotate point coordinates
        xr = cos_angle * xt + sin_angle * yt
        yr = -sin_angle * xt + cos_angle * yt
        
        # Calculate the distance from the point to the ellipse
        part1 = (xr / rx) ** 2
        part2 = (yr / ry) ** 2
        return (part1 + part2) ** 0.5

    # Calculate distances from all points to the ellipse and compute their variance
    distances = [point_to_ellipse_distance(x, y) for (x, y) in points]
    variance = np.var(distances)
    return variance


In [ ]:
from skimage.morphology import skeletonize
import itertools
from scipy.spatial.distance import pdist

def is_likely_antenna(contour, length, i, elongation_threshold=6, length_threshold=50):
    ### UNUSED (DOESNT WORK PROPERLY TO DETECT ANTENNA)
    ### INITIAL GOAL : DETERMINE IF A CONTOUR IS LIKELY AN ANTENNA BASED ON ITS ELONGATION AND LENGTH
    x, y, w, h = cv2.boundingRect(contour)
    elongation = max(w, h) / (min(w, h) if min(w, h) > 0 else 1)
    print('Dimension of bounding rect for contour {0}: [{1};{2}]'.format(i,w,h))
    print('Elongation : {0} | Length : {1}'.format(elongation,length))
    return (length > length_threshold and elongation > elongation_threshold), length

In [ ]:
def caculate_body_parts_features(row,mask, blockSize, ksize, k, threshold_coef, plot=False, img=None, bug_type=None):
    ### COMPUTE THE FEATURES OF THE BODY PARTS OF THE INSECT USING THE MASK OF THE INSECT
    ### 1 : FINDING THE BODY PARTS : SKELETONIZE THE MASK, APPLY HARRIS CORNER DETECTION, FIND CONTOURS IN THE SKELETON
    ### 2 : EXTRACTING STATISTICS : MEAN LENGTH, MAX LENGTH, MIN LENGTH, STD LENGTH
    ### 3 : EXTRACTING THE SPREAD OF BODY PARTS : MEAN DISTANCE BETWEEN CENTROIDS OF BODY PARTS

    start = process_time()
    print("Starting computation of body parts features")

    # Skeletonize the mask
    skeleton = skeletonize(mask.astype(bool)).astype(np.uint8) * 255

    # Apply Harris corner detection
    harris_corners = cv2.cornerHarris(skeleton, blockSize, ksize, k)
    corners_threshold = harris_corners > threshold_coef * harris_corners.max()
    skeleton[corners_threshold] = 0  # Optional: remove corners for clearer paths

    # Find contours in the skeleton
    contours, _ = cv2.findContours(skeleton, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Analyze each contour to determine if it's an antenna
    body_part_lengths = []
    #antennas = []
    centroids = []
    for i, contour in enumerate(contours):
        length = cv2.arcLength(contour, True)
        #if is_likely_antenna(contour, length, i): antennas.append(contour)
        body_part_lengths.append(length)
        cx = int(np.mean(contour[:, :, 0]))
        cy = int(np.mean(contour[:, :, 1]))
        centroids.append([cx, cy])

    # Calculate statistics
    n = len(contours)
    if n == 0:
        print("Warning : could not detect any body part") 
        return
    else:
        row['body_parts_mean_length'] = np.mean(body_part_lengths)
        row['body_parts_max_length'] = np.max(body_part_lengths)
        row['body_parts_min_length']  = np.min(body_part_lengths)
        row['body_parts_std_length'] = np.std(body_part_lengths)

    # Calculate mean distance between centroids
    if len(centroids) > 1:
        centroid_array = np.array(centroids)
        distances = pdist(centroid_array, 'euclidean')
        mean_distance = np.mean(distances)
        row['body_parts_spread'] = mean_distance
    else:
        print("Warning : Only one body part detected")

    # Visualization and Results
    if plot:
        #num_antennas = len(antennas)
        #print("Number of antennas: {0}".format(num_body_parts))
        plt.figure(figsize=(10, 5))
        plt.subplot(121)
        extra = "" if bug_type is None else " (" + bug_type + ")"
        if img is None:
            plt.imshow(mask, cmap='gray')
            plt.title('Original Picture' + extra)
        else:
            plt.imshow(img)
            plt.title('Original Mask' + extra)
        plt.axis('off')
        
        plt.subplot(122)
        plt.imshow(skeleton, cmap='gray')
        y, x = np.where(corners_threshold) 
        plt.scatter(x, y, color='white', s=0.5, marker='x', label="Corner", alpha=0.5)  # mark corners
        #for i, antenna in enumerate(antennas):
        #    label="Antenna" if i == 0 else None
        #    plt.plot(antenna[:, 0, 0], antenna[:, 0, 1], 'r', linewidth=8, label=label, alpha=0.3)  # plot antennas
        for i, contour in enumerate(contours):
            plt.plot(contour[:, 0, 0], contour[:, 0, 1], linewidth=2, label="Contour " + str(i))

        plt.legend()
        plt.title('Skeleton with Antennas Identified (n_centroids={0})'.format(len(centroids)))
        plt.axis('off')

        plt.tight_layout()
        plt.show()
    end = process_time()
    print("Computation took ", end-start)


In [ ]:
def calculate_axis_least_inertia(row,bug_argwhere_indices, centroid):
    ### COMPUTE THE AXIS OF LEAST INERTIA OF THE INSECT (INDICATOR OF THE ORIENTATION OF THE SHAPE)
    ### AXIS OF LEAST INERTIA : THE LINE FOR WHICH THE INTEGRAL OF THE SQUARE OF THE DISTANCES TO POINTS ON THE SHAPE BOUNDARY IS A MINIMUM
    ### X AXIS : THE EIGENVECTOR ASSOCIATED WITH THE SMALLEST EIGENVALUE OF THE COVARIANCE MATRIX OF THE SHIFTED POINTS
    ### Y AXIS : THE EIGENVECTOR ASSOCIATED WITH THE LARGEST EIGENVALUE OF THE COVARIANCE MATRIX OF THE SHIFTED POINTS

    start = process_time()
    print("Starting computation of axis of least inertia")
    # Shift points so centroid is at the origin
    shifted_points = bug_argwhere_indices - centroid
    
    # Compute covariance matrix of the shifted points
    covariance_matrix = np.cov(shifted_points, rowvar=False)
    
    # Perform eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    
    # Find the eigenvector associated with the smallest eigenvalue
    min_eigenvalue_index = np.argmin(eigenvalues)
    axis_least_inertia = eigenvectors[:, min_eigenvalue_index]
    row['axis_least_inertia_x'] = axis_least_inertia[0]
    row['axis_least_inertia_y'] = axis_least_inertia[1]
    end = process_time()
    print("Computation took ", end - start)


In [ ]:
def get_mask_compactness(mask_area, mask_perimeter):
    ### COMPUTE THE COMPACTNESS OF THE INSECT MASK (INDICATOR OF HOW CLOSE THE SHAPE IS TO A CIRCLE)
    ### SHOULD BE EQUAL TO THE ROUNDNESS FEATURE - WE'LL CHECK LATER
    ### COMPACTNESS = 4 * PI * AREA / PERIMETER^2
    ### RETURNS THE COMPACTNESS
    
    if mask_perimeter == 0:  # Avoid division by zero
        return 0
    compactness = 4 * np.pi * mask_area / (mask_perimeter ** 2)
    return compactness

#### REMOVING PROBLEMATIC ROW

In [ ]:
# There is an image number that doesn't have a file for the mask because it has 2 bugs, let's ignore it
print(154 in im_dict)
print(154 in mask_dict)
data.loc[154,'bug_type']

True
False


'Bee & Bumblebee'

In [ ]:
data = data.drop(154)

#### INITIALIZING THE COLUMNS FOR OUR FEATURES

In [ ]:
data = data.assign(nb_pixels_ratio=0, image_symmetry_index=0,mask_bb_symmetry_index=0, orthogonal_lines_ratio=0, roundness=0, 
                    mean_centroid_distance=0, std_centroid_distance=0, max_centroid_distance=0, min_centroid_distance=0,
                    skewness_centroid_distance=0, kurtosis_centroid_distance=0, aspect_ratio=0, mask_area=0, mask_perimeter=0, 
                    mask_compactness=0, hull_area=0, hull_to_insect_area_ratio=0, hull_convexity=0, 
                    hull_triangle_similarity=0, ellipse_angle=0, ellipse_axis_ratio=0, ellipse_eccentricity=0, ellipse_variance=0, 
                    axis_least_inertia_x=0, axis_least_inertia_y=0,rectangularity=0, body_parts_mean_length=0, body_parts_max_length=0,
                    body_parts_std_length=0, body_parts_spread=0, rest_entropy=0, mask_entropy=0)
colors = ['red', 'green', 'blue']
hsv = ['hue', 'saturation', 'value']
aggregates = ["min","max","mean","median","std","q1","q3"]
for color in colors + hsv:
    for aggregate in aggregates:
        data[color + "_mask_" + aggregate] = 0
        data[color + "_rest_" + aggregate] = 0

num_descriptors = 10
for i in range(num_descriptors):
    data[f'fourier_descriptor_real_{i}'] = 0
    data[f'fourier_descriptor_imag_{i}'] = 0
data = data.copy()

C:\Users\milen\AppData\Local\Temp\ipykernel_6272\589397231.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[color + "_rest_" + aggregate] = 0
C:\Users\milen\AppData\Local\Temp\ipykernel_6272\589397231.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[color + "_mask_" + aggregate] = 0
C:\Users\milen\AppData\Local\Temp\ipykernel_6272\589397231.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

#uncomment if you want to load data with the features already computed and compute a new feature
data = pd.read_csv("data/processed_data.csv",header=0,index_col='ID')
for col in data.columns:
    if data[col].sum() == 0:
        print(col + ' is full of 0s')

#### MAIN FUNCTION TO COMPUTE ALL FEATURES FROM THE IMAGES AND MASKS IN TRAIN FOLDER

In [ ]:
from scipy.spatial import ConvexHull


def compute_features(debug=True, plot_skeleton=False):
    #centroid_cols = [x for x in data.columns if "centroid" in x]

    #for index, row in data.iloc[:2].iterrows():
    for index, row in data.iterrows():
        print("Processing image ", index)
        num = index
        im_arr = im_dict[num]
        mask_arr = mask_dict[num]

        ## Step 1 : Preparation for feature computations (variable we will reuse)
        gray_im_arr = convert_to_grayscale(im_arr)
        mask_area = np.sum(mask_arr)
        is_mask = mask_arr == 1
        bug_indices = np.where(is_mask)
        non_bug_indices = np.where(~is_mask) #~mask_arr.astype(bool)
        bug_argwhere_indices = np.argwhere(is_mask)
        centroid = np.mean(bug_argwhere_indices, axis=0)
        edges = find_edges(mask_arr)
        edges_y, edges_x = np.where(edges == 1)
        
        # Extract points from the mask
        points = np.column_stack(bug_indices)
        
        if debug:
            print("Shape of the image array: ", im_arr.shape)
            print("Shape of the mask array: ", mask_arr.shape)
            print("Shape of bug points array :", bug_indices)
            #edges_opencv = find_edges_opencv(mask_arr)
            #edges_skimage = find_edges_skimage(mask_arr)
            #print("Shape of the opencv edges array: ", edges_opencv.shape)
            #print("Shape of the skimage edges array: ", edges_skimage.shape)
            #print(np.unique(edges_opencv), np.unique(edges_skimage))
            #print("Roundness is ",get_roundness(mask_area, edges))
            #print("Roundness of opencv is ",get_roundness(mask_area, edges_opencv))
            #print("Roundness of skimage is ", get_roundness(mask_area, edges_skimage))

        ## Step 2 : Bounding rectangle and skeleton features
        bb_minx, bb_maxx, bb_miny, bb_maxy = calculate_bounding_rectangle_features(row, mask_arr, mask_area)
        mask_arr_in_bb = mask_arr[bb_minx:bb_maxx+1, bb_miny:bb_maxy+1]
        row['orthogonal_lines_ratio'] = find_and_plot_orthogonal_lines(mask_arr_in_bb)
        if plot_skeleton:
            im_arr_in_bb = im_arr[bb_minx:bb_maxx+1, bb_miny:bb_maxy+1]
            caculate_body_parts_features(row,mask_arr_in_bb,20, 5, 0.2, 0.25, plot=plot_skeleton, img=im_arr_in_bb, bug_type=row.bug_type)
            continue
        else:
            caculate_body_parts_features(row,mask_arr_in_bb,20, 5, 0.2, 0.25)
        
        ## Step 3 : Default features (celles du pdf)
        row['nb_pixels_ratio'] = get_ratio(mask_area,gray_im_arr)
        for channel_id, channel_color in enumerate(colors):
            channel_arr = im_arr[:, :, channel_id]
            #get the color values only within the mask
            calculate_color_aggregates(row,channel_color + '_mask',channel_arr[bug_indices])
            calculate_color_aggregates(row,channel_color + '_rest', channel_arr[non_bug_indices])
        row['image_symmetry_index'] = get_symmetry_index(gray_im_arr)
        row['mask_bb_symmetry_index'] = get_symmetry_index(gray_im_arr[bb_minx:bb_maxx+1, bb_miny:bb_maxy+1])

        ## Step 4 : Additional features
        row['roundness'], mask_perimeter = get_roundness(mask_area, edges)
        #hsv_image = rgb_to_hsv(im_arr)
        #calculate_hsv_statistics(row, hsv_image, bug_indices, non_bug_indices)
        hsv_image2 = rgb_to_hsv_opencv(im_arr)
        #print(hsv_image2.shape,im_arr.shape,non_bug_indices[0].shape,non_bug_indices[1].shape)
        calculate_hsv_statistics(row, hsv_image2, bug_indices, non_bug_indices)
        
        row['rest_entropy'] = calculate_entropy(gray_im_arr[non_bug_indices])
        row['mask_entropy'] = calculate_entropy(gray_im_arr[bug_indices])
        row['mask_area'] = mask_area
        row['mask_perimeter'] = mask_perimeter
        row['mask_compactness'] =  get_mask_compactness(mask_area, mask_perimeter)
        edge_points = np.column_stack((edges_y, edges_x))
        calculate_geometric_centroid_distance_aggregates(row,mask_arr, bug_argwhere_indices, centroid, boundary_pixels=edge_points)
        #print(row[centroid_cols])
        #calculate_geometric_centroid_distance_aggregates(row,mask_arr, bug_argwhere_indices, centroid, plot='Manual')
        #print(row[centroid_cols])
        #row['fractal_dimension'] = calculate_fractal_dimension(mask_arr)
        calculate_fourier_descriptors(row,mask_arr, 10, edges_x, edges_y)
        calculate_axis_least_inertia(row,bug_argwhere_indices, centroid)
        calculate_ellipse_features(row, mask_arr, points)

        # Calculate convex hull properties
        hull = ConvexHull(points)
        calculate_convex_hull_properties(row, mask_arr, hull, mask_area, mask_perimeter)
        row['hull_triangle_similarity'] = get_triangle_similarity(hull, points, mask_area)

        ### Step 5 : Save the row
        data.loc[index]= row
    return data.head()
compute_features(plot_skeleton=False)

Processing image  1
Time taken for finding edges: 0.078125
Shape of the image array:  (4000, 6000, 3)
Shape of the mask array:  (4000, 6000)
Shape of bug points array : (array([1958, 1958, 1958, ..., 2485, 2485, 2485], dtype=int64), array([2822, 2823, 2824, ..., 3192, 3193, 3194], dtype=int64))
Starting computation of bounding rectangle features
Computation took  0.0
Processing image  2
Time taken for finding edges: 0.109375
Shape of the image array:  (4000, 6000, 3)
Shape of the mask array:  (4000, 6000)
Shape of bug points array : (array([1411, 1411, 1411, ..., 1962, 1962, 1962], dtype=int64), array([3238, 3239, 3240, ..., 3218, 3219, 3220], dtype=int64))
Starting computation of bounding rectangle features
Computation took  0.0
Processing image  3
Time taken for finding edges: 0.03125
Shape of the image array:  (4000, 6000, 3)
Shape of the mask array:  (4000, 6000)
Shape of bug points array : (array([ 620,  620,  620, ..., 1667, 1667, 1667], dtype=int64), array([3004, 3005, 3006, ...

,bug_type,species,nb_pixels_ratio,image_symmetry_index,mask_bb_symmetry_index,roundness,mean_centroid_distance,std_centroid_distance,max_centroid_distance,min_centroid_distance,skewness_centroid_distance,kurtosis_centroid_distance,aspect_ratio,mask_area,mask_perimeter,mask_compactness,hull_area,hull_to_insect_area_ratio,hull_convexity,hull_triangle_similarity,ellipse_angle,ellipse_axis_ratio,ellipse_eccentricity,ellipse_variance,axis_least_inertia_x,axis_least_inertia_y,rectangularity,body_parts_mean_length,body_parts_max_length,body_parts_std_length,body_parts_spread,rest_entropy,mask_entropy,red_mask_min,red_rest_min,red_mask_max,red_rest_max,red_mask_mean,red_rest_mean,red_mask_median,red_rest_median,red_mask_std,red_rest_std,red_mask_q1,red_rest_q1,red_mask_q3,red_rest_q3,green_mask_min,green_rest_min,green_mask_max,green_rest_max,green_mask_mean,green_rest_mean,green_mask_median,green_rest_median,green_mask_std,green_rest_std,green_mask_q1,green_rest_q1,green_mask_q3,green_rest_q3,blue_mask_min,blue_rest_min,blue_mask_max,blue_rest_max,blue_mask_mean,blue_rest_mean,blue_mask_median,blue_rest_median,blue_mask_std,blue_rest_std,blue_mask_q1,blue_rest_q1,blue_mask_q3,blue_rest_q3,hue_mask_min,hue_rest_min,hue_mask_max,hue_rest_max,hue_mask_mean,hue_rest_mean,hue_mask_median,hue_rest_median,hue_mask_std,hue_rest_std,hue_mask_q1,hue_rest_q1,hue_mask_q3,hue_rest_q3,saturation_mask_min,saturation_rest_min,saturation_mask_max,saturation_rest_max,saturation_mask_mean,saturation_rest_mean,saturation_mask_median,saturation_rest_median,saturation_mask_std,saturation_rest_std,saturation_mask_q1,saturation_rest_q1,saturation_mask_q3,saturation_rest_q3,value_mask_min,value_rest_min,value_mask_max,value_rest_max,value_mask_mean,value_rest_mean,value_mask_median,value_rest_median,value_mask_std,value_rest_std,value_mask_q1,value_rest_q1,value_mask_q3,value_rest_q3,fourier_descriptor_real_0,fourier_descriptor_imag_0,fourier_descriptor_real_1,fourier_descriptor_imag_1,fourier_descriptor_real_2,fourier_descriptor_imag_2,fourier_descriptor_real_3,fourier_descriptor_imag_3,fourier_descriptor_real_4,fourier_descriptor_imag_4,fourier_descriptor_real_5,fourier_descriptor_imag_5,fourier_descriptor_real_6,fourier_descriptor_imag_6,fourier_descriptor_real_7,fourier_descriptor_imag_7,fourier_descriptor_real_8,fourier_descriptor_imag_8,fourier_descriptor_real_9,fourier_descriptor_imag_9,orthogonal_lines_ratio
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Bee,Apis mellifera,0.00720,65.84252,70.05927,0.04671,245.17234,55.34848,362.54864,127.67198,-0.05793,-0.81783,0.81106,172773,6818,0.04671,265227.5,0.65141,0.28697,0.82641,4.10052,0.87838,0.47797,0.09994,-0.84075,0.54143,0.50264,434.17916,1566.46927,545.07894,252.45220,7.46081,6.96751,5,1,203,255,66.75052,107.40358,54,128,47.32937,61.07331,26,43,99,156,3,2,199,255,53.38703,105.37812,36,123,44.41560,58.06868,17,49,81,151,0,0,186,255,38.78619,84.57307,23,85,35.29870,57.19889,13,28,54,132,0,0,178,179,22.95534,42.77689,14,27,35.49440,46.07638,10,21,20,38,0,0,255,255,113.96383,66.07887,115.0,51,41.47447,50.35127,83,26,143,102,5,3,203,255,66.82641,109.17600,55,128,47.29504,59.73760,26,50,99,156,0.80235,0.59685,-0.02535,-0.00863,-0.01372,0.00637,-0.00183,-0.00426,-0.01064,0.00436,-0.00170,0.00806,-0.00169,0.00229,-0.00351,0.00471,-0.00095,0.00136,-0.00231,0.00099,0.67929
2,Bee,Apis mellifera,0.00831,68.89684,40.14367,0.04844,279.99803,85.16388,478.80295,108.00951,-0.06321,-0.61658,0.65248,199332,7191,0.04844,350945.5,0.56799,0.32091,0.99793,5.41676,0.56809,0.82297,0.09049,-0.96574,0.25950,0.42684,632.80009,4440.11845,1357.73020,350.97635,7.47115,6.81133,2,0,248,255,62.01555,101.10645,54,120,41.36733,61.73756,25,36,95,151,2,1,251,255,50.14088,98.70830,32,116,41.82911,58.44320,16,42,81,145,0,0,244,255,34.09561,74.86207,18,65,33.49315,56.03765,10,24,53,120,0,0,179,179,24.34423,40.25772,16,27,34.25677,43.17561,11,22,2

# Check coherence of results

In [ ]:
data.describe()

In [ ]:
train_data = pd.read_csv('data/processed_data.csv',header=0,index_col='ID')
train_data.describe()

In [ ]:
a

In [ ]:
data.to_csv("data/processed_test_data.csv")

NameError: name 'b' is not defined